### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 10, 15)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-10-15
yesterday: 2023-10-12 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 8)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2023-10-04
yesterday: 2023-10-12


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,TOA,2023-09-06,26.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'TOA'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,643,TOA,SETTHSI,33.50,36.50,25.00,42.37,5.38,"2,029.00","67,971.50",117.78,0.81,645,2017-10-13 05:51:19.667793,2023-01-31 03:08:01.315353


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "TOA" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,TOA,2023-10-12,21.90,22.10,21.70,"887,648",22.10
1,TOA,2023-10-11,22.00,22.10,21.50,"2,078,386",21.50
2,TOA,2023-10-10,21.40,21.80,21.20,"1,081,521",21.40
3,TOA,2023-10-09,21.50,22.10,21.10,"2,799,739",21.80
4,TOA,2023-10-06,22.00,22.40,21.80,"1,221,078",21.90


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2023-10-12' AND name IN ('AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'WHAIR', 'WHART')
ORDER BY name, date


30

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2023-10-04' AND name IN ('AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'WHAIR', 'WHART') 
ORDER BY name


30

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(30, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,AH,34.25,33.25,3.01
1,AIMIRT,10.40,10.50,-0.95
2,ASK,21.20,22.00,-3.64
3,ASP,2.82,2.80,0.71
4,AWC,3.68,3.64,1.10
5,BCH,20.20,20.20,0.00
6,CPNREIT,9.95,10.20,-2.45
7,DIF,9.05,9.05,0.00
8,GVREIT,5.75,5.80,-0.86
9,IVL,25.75,25.00,3.00


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
12,JMT,38.00,47.50,-20.00%,Worse
22,SINGER,11.00,12.20,-9.84%,Worse
11,JMART,21.00,22.30,-5.83%,Worse
2,ASK,21.20,22.00,-3.64%,Worse
24,SYNEX,11.20,11.50,-2.61%,Worse
6,CPNREIT,9.95,10.20,-2.45%,Worse
27,TOA,21.90,22.40,-2.23%,Worse
21,SENA,2.64,2.70,-2.22%,Worse
13,KCE,55.50,56.50,-1.77%,Worse
29,WHART,9.60,9.75,-1.54%,Worse


In [16]:
file_name = 'trend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(data_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [17]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
1,AIMIRT,10.40,10.50,-0.95,Worse
2,ASK,21.20,22.00,-3.64,Worse
6,CPNREIT,9.95,10.20,-2.45,Worse
8,GVREIT,5.75,5.80,-0.86,Worse
11,JMART,21.00,22.30,-5.83,Worse
12,JMT,38.00,47.50,-20.00,Worse
13,KCE,55.50,56.50,-1.77,Worse
14,MCS,6.65,6.70,-0.75,Worse
19,RCL,21.30,21.40,-0.47,Worse
21,SENA,2.64,2.70,-2.22,Worse


In [18]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,53.33%
Better,36.67%
No Change,10.00%


In [19]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(249, 8)

In [20]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [21]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(30, 13)

In [22]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
12,JMT,47.50,38.00,-20.00%,Worse,SET50
11,JMART,22.30,21.00,-5.83%,Worse,SET50
20,SCC,298.00,300.00,0.67%,Better,SET50
4,AWC,3.64,3.68,1.10%,Better,SET50
9,IVL,25.00,25.75,3.00%,Better,SET50
18,PTTGC,32.75,33.75,3.05%,Better,SET50


In [23]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,66.67%
Worse,33.33%


In [24]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
22,SINGER,12.20,11.00,-9.84%,Worse,SET100
13,KCE,56.50,55.50,-1.77%,Worse,SET100
19,RCL,21.40,21.30,-0.47%,Worse,SET100
5,BCH,20.20,20.20,0.00%,No Change,SET100
17,PTG,8.90,8.90,0.00%,No Change,SET100
16,ORI,10.00,10.30,3.00%,Better,SET100
23,STA,14.50,15.10,4.14%,Better,SET100


In [25]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,42.86%
No Change,28.57%
Better,28.57%


In [26]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
2,ASK,22.00,21.20,-3.64%,Worse,SET999
24,SYNEX,11.50,11.20,-2.61%,Worse,SET999
6,CPNREIT,10.20,9.95,-2.45%,Worse,SET999
27,TOA,22.40,21.90,-2.23%,Worse,SET999
21,SENA,2.70,2.64,-2.22%,Worse,SET999
29,WHART,9.75,9.60,-1.54%,Worse,SET999
1,AIMIRT,10.50,10.40,-0.95%,Worse,SET999
8,GVREIT,5.80,5.75,-0.86%,Worse,SET999
28,WHAIR,6.20,6.15,-0.81%,Worse,SET999
26,TMT,6.65,6.60,-0.75%,Worse,SET999


In [27]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,64.71%
Better,29.41%
No Change,5.88%


In [28]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [29]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
